# Working with salesdata.txt - Structured API

## Load data

Load the data in `sales/salesdata.txt` into an DataFrame:

In [1]:
df_sales = spark.read.csv('sales/salesdata.txt', sep='\t', inferSchema=True)
df_sales.printSchema()
df_sales.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)

+----------------+----------+-----------+------+----------+
|             _c0|       _c1|        _c2|   _c3|       _c4|
+----------------+----------+-----------+------+----------+
|2023-12-28 10:15|Mondonhedo|Moda hombre|185.82|  metalico|
|2022-02-26 19:30| A Corunha|   Farmacia|113.53|mastercard|
|2022-10-15 19:15| A Corunha|Moda hombre|341.95|      visa|
|2021-07-15 12:47|  Monforte| Automocion|239.83|      visa|
|2022-03-09 20:11|   Ourense|    Calzado| 52.98|  efectivo|
+----------------+----------+-----------+------+----------+
only showing top 5 rows



In [2]:
df_sales = spark.read \
    .option("sep", '\t') \
    .csv('sales/salesdata.txt') 
df_sales.printSchema()
df_sales.show(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)

+----------------+----------+-----------+------+----------+
|             _c0|       _c1|        _c2|   _c3|       _c4|
+----------------+----------+-----------+------+----------+
|2023-12-28 10:15|Mondonhedo|Moda hombre|185.82|  metalico|
|2022-02-26 19:30| A Corunha|   Farmacia|113.53|mastercard|
|2022-10-15 19:15| A Corunha|Moda hombre|341.95|      visa|
|2021-07-15 12:47|  Monforte| Automocion|239.83|      visa|
|2022-03-09 20:11|   Ourense|    Calzado| 52.98|  efectivo|
+----------------+----------+-----------+------+----------+
only showing top 5 rows



In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("data", StringType(), True),
    StructField("location", StringType(), True),
    StructField("category", StringType(), True),
    StructField("cost", DoubleType(), True),
    StructField("payment", StringType(), True)    
])

df_sales = spark.read.csv('sales/salesdata.txt', sep='\t', schema=schema)
df_sales.printSchema()
df_sales.show(5)

root
 |-- data: string (nullable = true)
 |-- location: string (nullable = true)
 |-- category: string (nullable = true)
 |-- cost: double (nullable = true)
 |-- payment: string (nullable = true)

+----------------+----------+-----------+------+----------+
|            data|  location|   category|  cost|   payment|
+----------------+----------+-----------+------+----------+
|2023-12-28 10:15|Mondonhedo|Moda hombre|185.82|  metalico|
|2022-02-26 19:30| A Corunha|   Farmacia|113.53|mastercard|
|2022-10-15 19:15| A Corunha|Moda hombre|341.95|      visa|
|2021-07-15 12:47|  Monforte| Automocion|239.83|      visa|
|2022-03-09 20:11|   Ourense|    Calzado| 52.98|  efectivo|
+----------------+----------+-----------+------+----------+
only showing top 5 rows



In [4]:
df_sales.describe().show()

+-------+----------------+---------+------------+------------------+--------+
|summary|            data| location|    category|              cost| payment|
+-------+----------------+---------+------------+------------------+--------+
|  count|        25000000| 25000000|    24916895|          24916895|24833771|
|   mean|            null|     null|        null|155.41418828549845|    null|
| stddev|            null|     null|        null| 289.1996948916661|    null|
|    min|2020-01-01 09:00|A Corunha|Alimentacion|              0.95|   bizum|
|    max|2023-12-31 20:59|     Vigo|        Ropa|           3148.63|    visa|
+-------+----------------+---------+------------+------------------+--------+



In [5]:
df_sales.summary().show()

+-------+----------------+---------+------------+------------------+--------+
|summary|            data| location|    category|              cost| payment|
+-------+----------------+---------+------------+------------------+--------+
|  count|        25000000| 25000000|    24916895|          24916895|24833771|
|   mean|            null|     null|        null|155.41418828549845|    null|
| stddev|            null|     null|        null| 289.1996948916662|    null|
|    min|2020-01-01 09:00|A Corunha|Alimentacion|              0.95|   bizum|
|    25%|            null|     null|        null|             30.19|    null|
|    50%|            null|     null|        null|              72.6|    null|
|    75%|            null|     null|        null|            160.92|    null|
|    max|2023-12-31 20:59|     Vigo|        Ropa|           3148.63|    visa|
+-------+----------------+---------+------------+------------------+--------+



In [7]:
# Get count of records
df_sales.count()

# Check for nulls in each column
from pyspark.sql.functions import col, count, when, isnan

df_sales.select([count(when(col(c).isNull(), c)).alias(c) for c in df_sales.columns]).show()

# Get distinct counts
df_sales.select([col(c) for c in df_sales.columns]).distinct().count()

# Distinct values per column
for column in df_sales.columns:
    print("{}: {}".format(
        column,
        df_sales.select(column).distinct().count()
    ))

+----+--------+--------+-----+-------+
|data|location|category| cost|payment|
+----+--------+--------+-----+-------+
|   0|       0|   83105|83105| 166229|
+----+--------+--------+-----+-------+

data: 1051910
location: 20
category: 21
cost: 275983
payment: 10


## Count the number of sales per Location

In [8]:
df_sales.groupBy('location').count().show()

+----------+-------+
|  location|  count|
+----------+-------+
|     Marin|1048705|
|    Ferrol|1309293|
| A Corunha|1962416|
|  Chantada| 784125|
|   Ribeira|1308946|
|       Foz| 652930|
|  Carballo|1309247|
|    Sarria| 915957|
|   Oleiros|1308475|
|      Lugo|1569254|
|     Naron|1309011|
|   Arteixo|1309270|
|Mondonhedo| 656003|
|   Ourense|1572445|
|      Vigo|1963813|
|Pontevedra|1569912|
|    Burela| 916669|
|    Cangas|1045658|
|  Monforte| 786080|
|  Santiago|1701791|
+----------+-------+



## Sum the total of sales per Category

In [9]:
df_sales.groupBy('category').sum().show()

+------------+--------------------+
|    category|           sum(cost)|
+------------+--------------------+
|    Mascotas| 7.584429399999973E7|
|   Cosmetica|1.2452601331999926E8|
|  Jardineria|1.0055476798000023E8|
|  Moda mujer|2.3703872598999965E8|
|        Cine| 1.210281420000001E7|
|        null|                null|
|      Libros|3.8224972030000106E7|
|Complementos|1.0490708236999942E8|
|    Deportes|2.1338050501000005E8|
|        Ropa|1.6964773458000016E8|
| Electronica|1.2624441484899988E9|
|      Musica| 5.562587021999992E7|
|  Ferreteria|1.3911944093000016E8|
|     Calzado| 1.629241953400009E8|
|       Bebes|1.6751150133999974E8|
|  Automocion| 4.095095070099965E8|
|Alimentacion|2.0506244590000074E7|
|    Farmacia|  5.14402004999998E7|
| Moda hombre|1.9768038597999948E8|
|    Juguetes| 8.520542355000019E7|
+------------+--------------------+
only showing top 20 rows



## Find the minimum sale per date (ordered by date)

In [ ]:
df_sales.createOrReplaceView("tiendas")

spark.sql('select date ')

In [10]:
from pyspark.sql.functions import substring

df_sales.withColumn('date', substring("data",1,10)).groupBy('date').min().orderBy('date',ascending=True).show()

+----------+---------+
|      date|min(cost)|
+----------+---------+
|2020-01-01|      1.0|
|2020-01-02|     1.17|
|2020-01-03|     1.01|
|2020-01-04|     1.01|
|2020-01-05|     1.04|
|2020-01-06|      1.0|
|2020-01-07|      1.0|
|2020-01-08|     1.05|
|2020-01-09|     0.96|
|2020-01-10|     1.04|
|2020-01-11|     1.02|
|2020-01-12|     1.03|
|2020-01-13|     1.06|
|2020-01-14|     1.05|
|2020-01-15|     1.01|
|2020-01-16|     1.03|
|2020-01-17|     0.99|
|2020-01-18|     1.03|
|2020-01-19|     1.01|
|2020-01-20|     0.97|
+----------+---------+
only showing top 20 rows



## Calculate de average of sales per category when payment method is cash

In [11]:
df_sales.select('payment').distinct().show()

+-------------+
|      payment|
+-------------+
|       paypal|
|         null|
|         cash|
|       cheque|
|transferencia|
|        bizum|
|   mastercard|
|     metalico|
|         visa|
|     efectivo|
+-------------+



In [12]:
from pyspark.sql.functions import avg

In [13]:
df_sales.filter(df_sales.payment.isin(['cash', 'metalico', 'efectivo'])) \
    .groupBy('category') \
    .agg(avg('cost').alias('avg_cost')) \
    .orderBy('avg_cost') \
    .show()

+------------+------------------+
|    category|          avg_cost|
+------------+------------------+
|        Cine| 9.725644451664005|
|Alimentacion|16.383041183827473|
|      Libros| 27.18371338166287|
|    Farmacia|34.395888713329484|
|      Musica| 37.99189976035834|
|    Mascotas|  49.4886747817107|
|    Juguetes| 56.12774776663495|
|  Jardineria| 65.41876226556793|
|Complementos| 69.54259463496108|
|  Ferreteria| 80.41413392156794|
|   Cosmetica|  82.7454321649516|
|       Bebes| 92.12014584545565|
|        Ropa| 94.59480219691577|
|       Hogar|106.83470334196866|
|     Calzado|106.97273763555948|
|    Deportes|107.08177714089662|
| Moda hombre|108.04427212805584|
|  Moda mujer|116.72306091331262|
|  Automocion|148.16625218783474|
| Electronica| 200.0971840695597|
+------------+------------------+

